In [1]:
from mmllm.module import InstructCell
import anndata
import numpy as np
from utils import unify_gene_features

# Cell type annotation

This example demonstrates the cell type annotation task using the **InstructCell** model. The objective is to annotate a single cell based on its gene expression profile and metadata, such as species, tissue, and sequencing method.

- **Input Data**: A single-cell dataset in **H5AD format** and a **gene vocabulary file**.
- **Metadata**: Metadata from the dataset (e.g., species, tissue, sequencing method) is extracted for annotation.
- **Prompt**: The prompt dynamically integrates metadata and the gene expression profile to guide the model.
- **Output**: The model predicts annotations for the single cell.

Below is the implementation:

In [2]:
# Load the pre-trained InstructCell-chat model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-chat") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/cell_type_annotation/He-2020-Liver/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
gene_counts = adata[k, :].X.toarray()
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata and gene expression profile
prompt = (
    "Can you help me annotate this single cell from a {species}? It was sequenced using {sequencing_method} and is derived from {tissue}. The gene expression profile is {input}. Thanks!"
)

# Use the model to generate predictions
for key, value in model.predict(
    prompt, 
    gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if value is not None:  
        print(f"{key}: {value}")

text: Based on the provided details, the single cell from the human sequenced using HiSeq X Ten System and derived from liver has been annotated as Macrophage.


In [ ]:
# Load the pre-trained InstructCell-instruct model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-instruct") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/cell_type_annotation/He-2020-Liver/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
gene_counts = adata[k, :].X.toarray()
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata and gene expression profile
prompt = (
    "Can you help me annotate this single cell from a {species}? It was sequenced using {sequencing_method} and is derived from {tissue}. The gene expression profile is {input}. Thanks!"
)

# Use the model to generate predictions
for key, value in model.predict(
    prompt, 
    gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if value is not None:  
        print(f"{key}: {value}")

text: Macrophage


# Conditional pseudo-cell generation

This example demonstrates the conditional pseudo-Cell generation task using the **InstructCell** model. The goal is to generate a synthetic gene expression profile based on provided metadata, such as cell type, species, tissue, and sequencing method.

- **Input Data**: A single-cell dataset in **H5AD format** and a **gene vocabulary file**.
- **Metadata**: Metadata from the dataset (e.g., cell type, species, tissue, sequencing method) is used to define the prompt.
- **Prompt**: The prompt dynamically incorporates metadata to guide the model in generating a conditional pseudo-cell profile.
- **Output**: The generated gene profile is ranked and displayed, highlighting the top 20 genes with the highest expression levels.

Below is the implementation:

In [4]:
# Load the pre-trained InstructCell model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-chat") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/conditional_pseudo_cell_generation/PBMC68K/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata
prompt = (
    "Hey, can you whip up a single-cell gene profile for the given specs: cell type is {cell_type}, species is {species}, tissue is {tissue}, and sequencing method is {sequencing_method}? "
)


for key, value in model.predict(
    prompt, 
    # gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if not isinstance(value, str): 
        value = '\n'.join(
            f"{gene_vocab[idx]}: {int(value[idx])}" for idx in np.argsort(-value)[: 20]
        )
    print(f"{key}:\n{value}")

/data/fangyin/anaconda3/envs/instructcell/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


text:
Sure, here is a single-cell gene profile based on your specifications: Cell Type: CD4+/CD45RA+/CD25- Naive T Species: human Tissue: peripheral blood Sequencing Method: 10xGenomics (GemCode Technology Platform)
cell:
RPL13: 83
MALAT1: 70
RPL13A: 64
TMSB4X: 48
RPL10: 47
B2M: 43
RPL18: 35
RPS18: 35
RPL18A: 35
RPS3A: 33
RPS4X: 31
RPL3: 31
RPS12: 29
RPS25: 27
RPS2: 27
FTL: 25
RPS6: 25
RPS3: 25
RPL7: 24
RPS19: 24


In [ ]:
# Load the pre-trained InstructCell-instruct model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-instruct") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/conditional_pseudo_cell_generation/PBMC68K/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata
prompt = (
    "Hey, can you whip up a single-cell gene profile for the given specs: cell type is {cell_type}, species is {species}, tissue is {tissue}, and sequencing method is {sequencing_method}? "
)


for key, value in model.predict(
    prompt, 
    # gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if not isinstance(value, str): 
        value = '\n'.join(
            f"{gene_vocab[idx]}: {int(value[idx])}" for idx in np.argsort(-value)[: 20]
        )
    print(f"{key}:\n{value}")

/data/fangyin/anaconda3/envs/instructcell/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


text:

cell:
MALAT1: 81
CD74: 41
RPL13A: 30
RPL18A: 22
RPL13: 22
RPS6: 18
RPL10: 17
RPS2: 16
RPL11: 14
RPS4X: 14
RPL7: 12
RPL12: 12
RPL19: 11
RPS18: 11
RPS27: 11
RPS19: 10
HLA-A: 10
MT-CO1: 9
MT-CO3: 9
TMSB4X: 9


We also support directly specifying and inputting the exact values for each placeholder in the prompt, as shown in this example：

In [6]:
# Load the pre-trained InstructCell-chat model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-chat") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/conditional_pseudo_cell_generation/PBMC68K/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its metadata
k = np.random.randint(0, len(adata)) 
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt
prompt = (
    "Hey, can you whip up a single-cell gene profile for the given specs: cell type is CD56+ NK, species is human, tissue is peripheral blood, and sequencing method is 10xGenomics (GemCode Technology Platform)? "
)

for key, value in model.predict(
    prompt, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if not isinstance(value, str): 
        value = '\n'.join(
            f"{gene_vocab[idx]}: {int(value[idx])}" for idx in np.argsort(-value)[: 20]
        )
    print(f"{key}:\n{value}")

/data/fangyin/anaconda3/envs/instructcell/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


text:
Sure, here's a single-cell gene profile based on your specifications: - Cell Type: CD56+ NK - Species: human - Tissue: peripheral blood - Sequencing Method: 10xGenomics (GemCode Technology Platform) The generated profile is as follows:
cell:
MALAT1: 43
B2M: 28
RPS3: 16
RPL10: 15
GNLY: 12
RPS2: 9
RPL21: 9
RPS6: 9
RPS16: 8
RPL13: 8
NKG7: 8
RPS27: 7
RPL14: 7
RPL7: 7
RPS12: 6
RPS4X: 6
HLA-C: 6
RPL3: 6
RPL28: 6
RPL19: 6


In [7]:
# Load the pre-trained InstructCell-instruct model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-instruct") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/conditional_pseudo_cell_generation/PBMC68K/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its metadata
k = np.random.randint(0, len(adata)) 
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt
prompt = (
    "Hey, can you whip up a single-cell gene profile for the given specs: cell type is CD56+ NK, species is human, tissue is peripheral blood, and sequencing method is 10xGenomics (GemCode Technology Platform)? "
)

for key, value in model.predict(
    prompt, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if not isinstance(value, str): 
        value = '\n'.join(
            f"{gene_vocab[idx]}: {int(value[idx])}" for idx in np.argsort(-value)[: 20]
        )
    print(f"{key}:\n{value}")

text:

cell:
B2M: 34
MALAT1: 28
TMSB4X: 27
RPS3: 22
RPL3: 19
RPS19: 18
CCL5: 17
RPL21: 17
GNLY: 15
RPS18: 14
RPL10: 12
RPL13: 11
RPS2: 10
RPS12: 10
RPL13A: 10
NKG7: 10
RPS9: 10
GZMB: 9
TMSB10: 8
MT-CO2: 8


/data/fangyin/anaconda3/envs/instructcell/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


# Drug sensitivity prediction

This example demonstrates the drug sensitivity prediction task using the **InstructCell** model. The goal is to predict how a single cell responds to a specific drug based on its gene expression profile and associated metadata.

- **Input Data**: A single-cell dataset in **H5AD format** and a **gene vocabulary file**.
- **Metadata**: Metadata such as species, tissue, drug, and sequencing method are dynamically incorporated into the prompt.
- **Prompt**: The prompt guides the model to predict drug sensitivity by providing the gene expression profile and metadata.
- **Output**: The model predicts the drug response.

Below is the implementation:

In [ ]:
# Load the pre-trained InstructCell-chat model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-chat") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/drug_sensitivity_prediction/GSE110894/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
gene_counts = adata[k, :].X.toarray()
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata and gene expression profile
prompt = (
    "Given {sequencing_method}, can you predict the response of the single cell {input} from {species} when exposed to {drug} in {tissue}?"
)

for key, value in model.predict(
    prompt, 
    gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if value is not None:  
        print(f"{key}: {value}")

text: The single cell from mouse when exposed to BET inhibitor (I-BET-762) in bone marrow is predicted to have Sensitive response.


In [9]:
# Load the pre-trained InstructCell-instruct model from HuggingFace
model = InstructCell.from_pretrained("zjunlp/InstructCell-instruct") 

# Load the single-cell data (H5AD format) and gene vocabulary file (numpy format)
adata = anndata.read_h5ad("./exp_log/demo_data/drug_sensitivity_prediction/GSE110894/rna.h5ad")
gene_vocab = np.load("./exp_log/gene_vocab.npy")
adata = unify_gene_features(adata, gene_vocab, force_gene_symbol_uppercase=False)

# Select a random single-cell sample and extract its gene counts and metadata
k = np.random.randint(0, len(adata)) 
gene_counts = adata[k, :].X.toarray()
sc_metadata = adata[k, :].obs.iloc[0].to_dict()

# Define the model prompt with placeholders for metadata and gene expression profile
prompt = (
    "Given {sequencing_method}, can you predict the response of the single cell {input} from {species} when exposed to {drug} in {tissue}?"
)

for key, value in model.predict(
    prompt, 
    gene_counts=gene_counts, 
    sc_metadata=sc_metadata, 
    do_sample=True, 
    top_p=0.95,
    top_k=50,
    max_new_tokens=256,
).items():
    if value is not None:  
        print(f"{key}: {value}")

text: Sensitive
